In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from datasets import load_dataset, load_dataset_builder

/Users/user/mambaforge/envs/zoobot39_dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import sys
sys.path.append('../..')

from baselines.galaxy10_decals.dataset_wrapper import DatasetWrapper

In [4]:
dataset_loc = 'mwalmsley/galaxy10_decals'

In [5]:
# Load Dataset Builders 
decals10_builder = load_dataset_builder(dataset_loc, 'astropile', trust_remote_code=True)
# decals10_builder.set_format('torch')

In [6]:
type(decals10_builder)

datasets_modules.datasets.mwalmsley--galaxy10_decals.f7f0215e9b20abfd8f74854d9ac987a9e7ef308f4a8681fa7840260fba92924c.galaxy10_decals.GZDecals

In [7]:
decals10_train = load_dataset('mwalmsley/galaxy10_decals', 'astropile', split='train', trust_remote_code=True)
decals10_test = load_dataset('mwalmsley/galaxy10_decals', 'astropile', split='test', trust_remote_code=True)

galaxy10_decals
galaxy10_decals


In [8]:

decals10_train.set_format('torch')
decals10_test.set_format('torch')

In [9]:
len(decals10_train)

200

In [10]:
decals10_train[0]['label']

tensor(5)

In [11]:
decals10_train[0]['images_by_band']['band']

['g', 'r', 'z']

In [12]:
decals10_train[0]['images_by_band']['array'].shape

torch.Size([3, 224, 224])

In [13]:
type(decals10_train)

datasets.arrow_dataset.Dataset

In [14]:
wrapper = DatasetWrapper(
    train_dataset=decals10_train,
    test_dataset=decals10_test,
    feature_flag='images_by_band.array',
    label_flag='label',
    loading='iterated'
)

In [15]:
# decals10_train['images_by_band']

In [16]:
# wrapper.prepare_data()
# wrapper.feature_mean, wrapper.feature_std

In [17]:
train_loader = wrapper.train_dataloader()
val_loader = wrapper.val_dataloader()
test_loader = wrapper.test_dataloader()

AttributeError: 'DatasetWrapper' object has no attribute 'val_dataset'

In [ ]:
for images, labels in train_loader:
    print(images.shape, labels.shape)

torch.Size([128, 3, 224, 224]) torch.Size([128])
torch.Size([32, 3, 224, 224]) torch.Size([32])


In [ ]:
im_np = images[0].numpy().transpose(1, 2, 0)
im_np.min(), im_np.max()

(-0.1924183, 1.6967673)

In [ ]:
from model import SmallConvModel

config = {
    'channels': 3,
    'img_size': 224,
    'layers':
        [
            {'in_channels': 3, 'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 0},
            {'in_channels': 32, 'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 0},
            {'in_channels': 32, 'out_channels': 32, 'kernel_size': 3, 'stride': 1, 'padding': 0},
        ],
    'representation_dim': 32*26*26,
    'head_size': 64,
    'num_classes': 10
}


model = SmallConvModel(config)

In [ ]:
trainer = pl.Trainer(max_epochs=1, accelerator='cpu')
# , fast_dev_run=True)

# TODO need to grab multi-class loss from somewhere and throw in
trainer.fit(model, wrapper)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Computing statistics for images_by_band.array: 100%|██████████| 1/1 [00:10<00:00, 10.40s/it]

  | Name           | Type               | Params
------------------------------------------------------
0 | encoder        | Sequential         | 19.4 K
1 | head           | Sequential         | 1.4 M 
2 | model          | Sequential         | 1.4 M 
3 | get_loss       | BCELoss            | 0     
4 | train_accuracy | MulticlassAccuracy | 0     
5 | val_accuracy   | MulticlassAccuracy | 0     
------------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.616     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: all elements of target should be between 0 and 1

In [ ]:
# alternative non-lightning loading option

In [ ]:
# target_size = 64
# from torchvision.transforms import Compose, Normalize, Resize, Lambda, RandomHorizontalFlip, RandomVerticalFlip


# normalize = Normalize(mean=1., std=1.)
#                     #   https://pytorch.org/vision/main/generated/torchvision.transforms.Lambda.html)

# _transforms = Compose(
#     [
#         Resize(target_size),
#         RandomHorizontalFlip(),
#         RandomVerticalFlip(),
#         Lambda(func=lambda x: np.arcsinh(x)),
#         normalize
#     ]
# )

# def transforms(examples):
#     examples["pixel_values"] = [_transforms(image.convert("RGB")) for image in examples["image"]]
#     return examples